# One Shot Learning using Siamese networks on Omniglot Dataset

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import sys, os, time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

## Loading Train Dataset

In [2]:
data_folder = './images_background/'
save_path = './data/'

In [4]:
X = []
y = []
lang_dict = {}
character_count = 0

for alphabet in os.listdir(data_folder):
    
    # print("loading alphabet: " + alphabet)
    lang_dict[alphabet] = [character_count, None]
    alphabet_path = os.path.join(data_folder, alphabet)
    
    for letter in os.listdir(alphabet_path):
        character_images = []
        character_path = os.path.join(alphabet_path, letter)
        # read all the images for the current character
        for filename in os.listdir(character_path):
            image_path = os.path.join(character_path, filename)
            image = plt.imread(image_path)
            character_images.append(image)
            y.append(character_count)
        try:
            X.append(np.stack(character_images))
        # edge case - last one
        except ValueError as e:
            print(e)
            print("error - category_images:", character_images)
        character_count += 1
    lang_dict[alphabet][1] = character_count - 1
    
y = np.vstack(y)
X = np.stack(X)
lang_dict

{'Alphabet_of_the_Magi': [0, 19],
 'Anglo-Saxon_Futhorc': [20, 48],
 'Arcadian': [49, 74],
 'Armenian': [75, 115],
 'Asomtavruli_(Georgian)': [116, 155],
 'Balinese': [156, 179],
 'Bengali': [180, 225],
 'Blackfoot_(Canadian_Aboriginal_Syllabics)': [226, 239],
 'Braille': [240, 265],
 'Burmese_(Myanmar)': [266, 299],
 'Cyrillic': [300, 332],
 'Early_Aramaic': [333, 354],
 'Futurama': [355, 380],
 'Grantha': [381, 423],
 'Greek': [424, 447],
 'Gujarati': [448, 495],
 'Hebrew': [496, 517],
 'Inuktitut_(Canadian_Aboriginal_Syllabics)': [518, 533],
 'Japanese_(hiragana)': [534, 585],
 'Japanese_(katakana)': [586, 632],
 'Korean': [633, 672],
 'Latin': [673, 698],
 'Malay_(Jawi_-_Arabic)': [699, 738],
 'Mkhedruli_(Georgian)': [739, 779],
 'N_Ko': [780, 812],
 'Ojibwe_(Canadian_Aboriginal_Syllabics)': [813, 826],
 'Sanskrit': [827, 868],
 'Syriac_(Estrangelo)': [869, 891],
 'Tagalog': [892, 908],
 'Tifinagh': [909, 963]}

### Train Validation split

In [5]:
split = 673
X_train, X_val = X[:split], X[split:]

train_classes = {}
for alphabet in list(lang_dict.keys())[:21]:
    train_classes[alphabet] = lang_dict[alphabet]
    
val_classes = {}
for alphabet in list(lang_dict.keys())[21:]:
    val_classes[alphabet] = lang_dict[alphabet]

In [6]:
X_train.shape, X_val.shape

((673, 20, 105, 105), (291, 20, 105, 105))

## Defining Helper Functions

In [7]:
def initialize_weights(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [8]:
def initialize_bias(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [9]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [10]:
def get_batch(batch_size,s="train"):
    """Create batch of n pairs, half same class, half different class"""
    if s == 'train':
        X = X_train
        categories = train_classes
    else:
        X = X_val
        categories = val_classes
    n_classes, n_examples, w, h = X.shape

    # randomly sample several classes to use in the batch
    categories = rng.choice(n_classes,size=(batch_size,),replace=False)
    
    # initialize 2 empty arrays for the input image batch
    pairs = [np.zeros((batch_size, h, w, 1)) for i in range(2)]
    
    # initialize vector for the targets
    targets = np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
    
    return pairs, targets

In [11]:
def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [12]:
def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = X_train
        categories = train_classes
    else:
        X = X_val
        categories = val_classes
    n_classes, n_examples, w, h = X.shape
    
    indices = rng.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low,high),size=(N,),replace=False)

    else: # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,),replace=False)            
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

In [13]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

## Creating the Network

In [ ]:
model = get_siamese_model((105, 105, 1))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 105, 105, 1)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 4096)         38947648    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None

In [ ]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

## Training the network

In [ ]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 20000 # No. of training iterations
N_way = 10 # how many classes for testing one-shot tasks
n_val = 100 # how many one-shot tasks to validate on
best = -1

In [ ]:
model_path = './model_evaluation/'

In [ ]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------
Instructions for updating:
Use tf.cast instead.

 ------------- 

Time for 200 iterations: 0.7782549897829691 mins
Train Loss: 2.554734230041504
Evaluating model on 100 random 10 way one-shot learning tasks ... 

Got an average of 49.0% 10 way one-shot learning accuracy 

Current best: 49.0, previous best: -1

 ------------- 

Time for 400 iterations: 1.4364153504371644 mins
Train Loss: 1.8151233196258545
Evaluating model on 100 random 10 way one-shot learning tasks ... 

Got an average of 59.0% 10 way one-shot learning accuracy 

Current best: 59.0, previous best: 49.0

 ------------- 

Time for 600 iterations: 2.087701916694641 mins
Train Loss: 1.45762300491333
Evaluating model on 100 random 10 way one-shot learning tasks ... 

Got an average of 65.0% 10 way one-shot learning accuracy 

Current best: 65.0, previous best: 59.0

 ------------- 

Time for 800 iterations: 2.7410284280776978 mins
Train Loss: 1.17247247695922

## Loading Test Dataset

In [ ]:
test_data = np.load('test.npy')
test_data.shape

(200, 2, 10, 105, 105, 1)

## Predicting for Test Data

In [ ]:
# Choose weights with best accuracy
model.load_weights(os.path.join(model_path, "weights.19200.h5"))

In [ ]:
predictions = []
for test_sample in in range(test_data):
    probabilities = model.predict([test_sample[0], test_sample[1]])
    predictions.append(np.argmax(probabilities))
predictions

[5,
 5,
 8,
 8,
 0,
 3,
 9,
 4,
 3,
 5,
 9,
 9,
 6,
 2,
 8,
 8,
 9,
 1,
 3,
 5,
 2,
 5,
 5,
 3,
 2,
 4,
 5,
 3,
 5,
 0,
 6,
 2,
 7,
 4,
 1,
 9,
 5,
 5,
 1,
 9,
 6,
 0,
 6,
 4,
 4,
 3,
 5,
 7,
 0,
 8,
 0,
 9,
 5,
 9,
 5,
 7,
 5,
 9,
 3,
 9,
 7,
 3,
 2,
 4,
 4,
 5,
 1,
 3,
 9,
 6,
 6,
 7,
 8,
 6,
 9,
 8,
 1,
 7,
 5,
 1,
 6,
 3,
 2,
 2,
 1,
 0,
 4,
 6,
 5,
 3,
 2,
 2,
 3,
 5,
 0,
 6,
 4,
 3,
 4,
 1,
 8,
 1,
 0,
 7,
 5,
 0,
 7,
 0,
 3,
 6,
 4,
 4,
 6,
 5,
 3,
 2,
 5,
 0,
 0,
 2,
 7,
 9,
 6,
 2,
 1,
 2,
 6,
 9,
 2,
 7,
 9,
 3,
 1,
 1,
 3,
 5,
 7,
 6,
 9,
 4,
 8,
 6,
 5,
 9,
 2,
 5,
 2,
 6,
 3,
 1,
 4,
 4,
 7,
 0,
 8,
 4,
 2,
 8,
 8,
 7,
 8,
 3,
 2,
 1,
 8,
 0,
 5,
 1,
 5,
 3,
 7,
 5,
 9,
 5,
 5,
 4,
 0,
 1,
 6,
 9,
 1,
 9,
 9,
 5,
 4,
 0,
 1,
 2,
 2,
 0,
 5,
 1,
 2,
 6,
 1,
 9,
 9,
 1,
 7,
 4]

In [ ]:
from collections import OrderedDict
output_dict = OrderedDict()
output_dict['Id'] = list(range(len(test_data)))
output_dict['Category'] = predictions
output_df = pd.DataFrame(data = output_dict)
output_df.to_csv("output.csv",index=False)